day 1

In [ ]:
from queue import PriorityQueue

f = open('1_2.txt')
lines = f.read().split('\n')

max_sum, cur_sum = 0, 0
q = PriorityQueue()
for line in lines:
  if line != '':
    cur_sum += int(line)
  else:
    max_sum = max(max_sum, cur_sum)
    q.put(-cur_sum)
    cur_sum = 0

top_three_sum = 0
for i in range(3):
  top_three_sum -= q.get()

max_sum, top_three_sum

(69528, 206152)

day 2

In [ ]:
shape_to_score = {'X': 1, 'Y': 2, 'Z': 3}
opp_to_shape = {'A': 'X', 'B': 'Y', 'C': 'Z'}

In [ ]:
def parse_rounds():
  s = open('2_2.txt').read()
  rounds = []

  for line in s.split('\n')[:-1]:#[1:-1]:
    tokens = line.split(' ')
    opp, you = tokens[0], tokens[1]
    rounds.append([opp_to_shape[opp], you])
  return rounds

In [ ]:
def eval_total_score(rounds):
  # key => [draws with, beats, loses]
  lookup = {
      'X': ['X', 'Z', 'Y'],
      'Y': ['Y', 'X', 'Z'],
      'Z': ['Z', 'Y', 'X'],
  }

  total_score = 0
  for opp, you in rounds:
    draws_with, beats, loses = lookup[opp]
    if draws_with == you: # opp and you draw with each other
      total_score += shape_to_score[you] + 3
    elif beats == you: # opp beats, you lose
      total_score += shape_to_score[you]
    elif loses == you: # opp loses, you beat
      total_score += shape_to_score[you] + 6
  return total_score

eval_total_score(parse_rounds())

14163

In [ ]:
result_to_score = {
    'X': 0, # you lose
    'Y': 3, # draw
    'Z': 6, # you win
}

def get_move_for_result(opp, result):
  lookup = {
    'X': ['X', 'Z', 'Y'],
    'Y': ['Y', 'X', 'Z'],
    'Z': ['Z', 'Y', 'X'],
  }
  options = lookup[opp]
  if result == 'Y': # draw
    return options[0]
  elif result == 'X': # you lose
    return options[1]
  elif result =='Z': # you win
    return options[2]

def eval_total_score_2(rounds):
  total_score = 0
  for opp, result in rounds:
    you = get_move_for_result(opp, result)
    total_score += shape_to_score[you] + result_to_score[result]
  return total_score

eval_total_score_2(parse_rounds())

12091

day 3

In [ ]:
def parse(fn):
  sacks = []
  with open(fn) as fd:
    for line in fd.readlines():
      line = line.strip()
      length = len(line) // 2
      left, right = line[:length], line[length:]
      sacks.append([left, right])
  return sacks

In [ ]:
def priority(char):
  if 'a' <= char and char <= 'z':
    return ord(char) - 96
  if 'A' <= char and char <= 'Z':
    return ord(char) - ord('A') + 27

def p1(sacks):
  priorities = 0
  for (left, right) in sacks:
    common_char = set(left).intersection(set(right))
    common_char = next(iter(common_char))
    priorities += priority(common_char)
  return priorities

p1(parse('3_2.txt'))

7863

In [ ]:
def p2(sacks):
  ctr = 0
  groups, group = [], []
  for i, (left, right) in enumerate(sacks):
    group.append(left + right)
    ctr += 1
    if ctr == 3:
      groups.append(group)
      ctr, group = 0, []

  priorities = 0
  for group in groups:
    common_char = set(group[0]).intersection(set(group[1])).intersection(set(group[2]))
    common_char = next(iter(common_char))
    priorities += priority(common_char)
  return priorities

p2(parse('3_2.txt'))

2488

day 4

In [ ]:
def parse(fn):
  range_pairs = []
  with open(fn) as fd:
    lines = fd.read().split('\n')[:-1]
    for line in lines:
      p1, p2 = line.split(',')
      p1_1, p1_2 = list(map(lambda x: int(x), p1.split('-')))
      p2_1, p2_2 = list(map(lambda x: int(x), p2.split('-')))
      range_pairs.append(((p1_1, p1_2), (p2_1, p2_2)))
  return range_pairs

In [ ]:
def overlaps(p1_1, p1_2, p2_1, p2_2):
  return max(p1_1, p2_1) <= min(p1_2, p2_2)

def p1(range_pairs):
  out = 0
  for ((p1_1, p1_2), (p2_1, p2_2)) in range_pairs:
    if overlaps(p1_1, p1_2, p2_1, p2_2):
      out += 1
  return out

p1(parse('4_2.txt'))

883

day 5

In [ ]:
import os
import re

def parse(fn):
  BLANK_LINE = os.linesep + os.linesep
  stacks_str, moves_str = open(fn).read().split(BLANK_LINE)
  n_stacks = int(stacks_str[-2]) # assumes n_stacks <= 9
  rows, moves = [], []

  for stack_row_str in stacks_str.split('\n')[:-1]:
    rows.append(stack_row_str
      .replace('] [', ',').replace('[', '').replace(']', '')
      .replace(' '*4, ',')
      .replace(' ', ',')
      .split(',')
    )

  for move_str in moves_str.split('\n')[:-1]:
    move = re.findall('move ([0-9]+) from ([0-9]+) to ([0-9]+)', move_str)
    moves.append([int(x) for x in move[0]])

  stacks = list([
    list(filter(lambda crate: crate != '', col)) for col in zip(*rows)
  ])

  return stacks, moves

In [ ]:
def p1(stacks, moves):
  for (n_to_move, from_idx, to_idx) in moves:
    for _ in range(n_to_move):
      stacks[to_idx - 1] = [stacks[from_idx - 1].pop(0)] + stacks[to_idx - 1]
  return ''.join([stack[0] for stack in stacks])

stacks, moves = parse('5_2.txt')
print(p1(stacks, moves))

RFFFWBPNS


In [ ]:
def p1(stacks, moves):
  for (n_to_move, from_idx, to_idx) in moves:
    stacks[to_idx - 1] = stacks[from_idx - 1][:n_to_move] + stacks[to_idx - 1]
    stacks[from_idx - 1] = stacks[from_idx - 1][n_to_move:]
  return ''.join([stack[0] for stack in stacks])

stacks, moves = parse('5_2.txt')
print(p1(stacks, moves))

CQQBBJFCS


day 6

In [ ]:
def parse(fn):
  return open(fn).read().strip()

def p1(s):
  substr = list(s[:4])
  for i, c in enumerate(s[4:]):
    substr_set = set(substr)

    if len(substr_set) == len(substr):
      return i + 4

    delete_c = substr.pop(0)
    substr.append(c)

  if len(substr_set) == len(substr):
    return i + 1
  return ''

p1('bvwbjplbgvbhsrlpgdmjqwftvncz')

5

In [ ]:
p1('nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg')

10

In [ ]:
p1('mjqjpqmgbljsphdztnvjfqwrcgsmlb')

7

In [ ]:
p1('zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw')

11

In [ ]:
p1(parse('6_2.txt'))

1262

In [ ]:
def p2(s):
  substr = list(s[:14])
  for i, c in enumerate(s[14:]):
    substr_set = set(substr)
    if len(substr_set) == len(substr):
      return i + 14
    substr.pop(0)
    substr.append(c)

  return ''

p2(parse('6_2.txt'))

3444

day 7

In [ ]:
def parse(fn):
  root = {} # the filesystem
  stk = [] # stack representing current working directory path

  def add_node(name, is_dir, size=-1):
    cur_node = root # traversal
    for node in stk:
      if node not in cur_node:
        cur_node[node] = {}
      cur_node = cur_node[node]

    if is_dir:
      cur_node[name] = {} # shitty duck typing
    else:
      cur_node[name] = (','.join(stk), size)

  for line in open(fn).read().split('$ ')[1:]:
    lines = line.split('\n')[:-1]

    # cd up one dir
    if lines[0] == 'cd ..':
      stk.pop()

    # cd into an existing dir
    elif lines[0][:2] == 'cd':
      stk.append(lines[0][3:])

    # update fs with new known dirs and files
    elif lines[0][:2] == 'ls':
      for line in lines[1:]:

        # new known dir
        if line[:3] == 'dir':
          add_node(line[4:], True)

        # new known files
        else:
          size, name = line.split(' ')
          add_node(name, False, int(size))

  return root

In [ ]:
from collections import defaultdict

def p1(root):
  dir_to_size = defaultdict(int)
  stk = [root]
  while stk:
    node = stk.pop()
    if isinstance(node, dict):
      stk.extend(node.values())
    else:
      comma_delimited_path, size = node
      ancestors = comma_delimited_path.split(',')
      for i in range(len(ancestors)):
        dir_to_size[','.join(ancestors[:i+1])] += size

  result, total_size = 0, dir_to_size['/']
  for dir, size in dir_to_size.items():
    if size <= 1e5:
      result += size

  return dir_to_size, total_size, result

p1(parse('7_2.txt'))[2]

1307902

In [ ]:
def p2(root):
  dir_to_size, total_size, _ = p1(root)
  candidates = [] # list of [(dir: str, size: int)]

  MIN_TOTAL_SIZE = total_size - 4e7 # to delete
  for dir, size in dir_to_size.items():
    if size >= MIN_TOTAL_SIZE:
      candidates.append((dir, size))

  candidates = sorted(candidates, key=lambda tup: tup[1])
  return candidates[0][1]

p2(parse('7_2.txt'))

7068748

day 8

In [ ]:
def parse(fn):
  out = []
  for line in open(fn).read().split('\n')[:-1]:
    out.append(list(map(lambda c: int(c), line)))
  return out

parse('8_1.txt')

[[3, 0, 3, 7, 3],
 [2, 5, 5, 1, 2],
 [6, 5, 3, 3, 2],
 [3, 3, 5, 4, 9],
 [3, 5, 3, 9, 0]]

In [ ]:
def is_visible(x, y, grid):
  h = grid[x][y]
  up = all([grid[row][y] < h for row in range(0, x)])
  down = all([grid[row][y] < h for row in range(x + 1, len(grid))])
  left = all([grid[x][col] < h for col in range(0, y)])
  right = all([grid[x][col] < h for col in range(y + 1, len(grid[0]))])
  return up or down or left or right

def p1(grid):
  out = (len(grid) * 2) + ((len(grid[0]) - 2) * 2)
  for row in range(1, len(grid) - 1):
    for col in range(1, len(grid[0]) - 1):
      if is_visible(row, col, grid):
        out += 1
  return out

p1(parse('8_2.txt'))

1794

In [ ]:
def viewing_distance(ox, oy, dx, dy, grid):
  x, y = ox + dx, oy + dy
  out = 0
  while x >= 0 and x < len(grid) and y >= 0 and y < len(grid[0]):
    out += 1
    if grid[x][y] >= grid[ox][oy]:
      return out # we can't see past this tree
    x += dx
    y += dy
  return out

def scenic_score(x, y, grid):
  product = 1
  for dx, dy in [(-1, 0), (0, 1), (0, -1), (1, 0)]:
    product *= viewing_distance(x, y, dx, dy, grid)
  return product

def p2(grid):
  max_ss = 0
  for row in range(len(grid)):
    for col in range(len(grid[0])):
      max_ss = max(max_ss, scenic_score(row, col, grid))
  return max_ss

p2(parse('8_2.txt'))

199272

day 9

In [ ]:
def parse(fn):
  moves = open(fn).read().split('\n')[:-1]
  moves = list(map(lambda move: move.split(' '), moves))
  return [(dir, int(magnitude)) for dir, magnitude in moves]

In [ ]:
from collections import defaultdict

dir_dxdy = {
  'R': (0, 1),
  'L': (0, -1),
  'U': (-1, 0),
  'D': (1, 0),
}

# update the tail
# If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:
# Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up
def update(hd, tl):
  hx, hy = hd
  tx, ty = tl
  if abs(hx - tx) <= 1 and abs(hy - ty) <= 1: # head, tail touching
    return tx, ty
  if hx == tx and hy > ty: # head to the right of tail
    return tx, ty + 1
  elif hx == tx and hy < ty: # head to the left of tail
    return tx, ty - 1
  elif hx > tx and hy == ty: # head below tail
    return tx + 1, ty
  elif hx < tx and hy == ty: # head above tail
    return tx - 1, ty
  elif hx < tx and hy < ty: # head NW of tail
    return tx - 1, ty - 1
  elif hx < tx and hy > ty: # head NE of tail
    return tx - 1, ty + 1
  elif hx > tx and hy < ty: # head SW of tail
    return tx + 1, ty - 1
  elif hx > tx and hy > ty: # head SE of tail
    return tx + 1, ty + 1

def p1(moves):
  hd, tl = (0, 0), (0, 0)
  freqs = defaultdict(int)

  for dir, magnitude in moves:
    dx, dy = dir_dxdy[dir]
    for _ in range(magnitude):
      hd = (hd[0] + dx, hd[1] + dy)
      tl = update(hd, tl)
      freqs[tl] += 1
  return len(freqs)

p1(parse('9_2.txt'))

5683

In [ ]:
def p2(moves):
  hd, tl = (0, 0), (0, 0)
  freqs = defaultdict(int)
  body = [hd] + [(0, 0) for i in range(9)] # len(body) <= 9, body[i] = position of i-th body part

  for dir, magnitude in moves:
    dx, dy = dir_dxdy[dir]
    for _ in range(magnitude):
      body[0] = (body[0][0] + dx, body[0][1] + dy)

      i = 0
      while i < len(body) - 1:
        b1, b2 = body[i], body[i + 1]
        body[i + 1] = update(b1, b2)
        i += 1
      freqs[body[9]] += 1

  return len(freqs)

p2(parse('9_2.txt'))

2372

day 10

In [ ]:
def parse(fn):
  instrs = open(fn).read().split('\n')[:-1]
  instrs = list(map(lambda row: row.split(' '), instrs))
  return [(row[0], int(row[1])) if len(row) > 1 else (row[0], 0) for row in instrs] # ignore noop

def p1(instrs):
  x, cycle = 1, 1
  total_ss = 0
  cycle_to_exec, incr_to_exec = 0, 0

  while instrs or cycle_to_exec != 0 or incr_to_exec != 0:
    if cycle_to_exec == 0:
      (instr, incr) = instrs.pop(0)
      if instr == 'addx':
        cycle_to_exec = cycle + 1
        incr_to_exec = incr

    # it matters where you put this!
    if cycle == 20 or (cycle >= 40 and (cycle - 20) % 40 == 0):
      total_ss += x * cycle

    if cycle_to_exec == cycle:
      x += incr_to_exec
      cycle_to_exec, incr_to_exec = 0, 0

    cycle += 1

  return total_ss

p1(parse('10_2.txt'))

14320

In [ ]:
import os

def p2(instrs):
  x = 1
  cycle_to_exec, incr_to_exec = 0, 0
  s = ''

  for cycle in range(1, 241):
    if (cycle - 1) % 40 == 0:
      print(s)
      s = ''

    if cycle_to_exec == 0:
      (instr, incr) = instrs.pop(0)
      if instr == 'addx':
        cycle_to_exec, incr_to_exec = cycle + 1, incr

    # sprite positioning
    pos = (cycle - 1) % 40 # nasty
    if x >= pos - 1 and x <= pos + 1:
      s += ' #'
    else:
      s += ' .'

    if cycle_to_exec == cycle:
      x += incr_to_exec
      cycle_to_exec, incr_to_exec = 0, 0

  print(s)

p2(parse('10_2.txt'))


 # # # . . . # # . . # # # . . # # # . . # . . # . . # # . . # # # . . . . # # .
 # . . # . # . . # . # . . # . # . . # . # . # . . # . . # . # . . # . . . . # .
 # . . # . # . . . . # . . # . # # # . . # # . . . # . . # . # . . # . . . . # .
 # # # . . # . . . . # # # . . # . . # . # . # . . # # # # . # # # . . . . . # .
 # . . . . # . . # . # . . . . # . . # . # . # . . # . . # . # . . . . # . . # .
 # . . . . . # # . . # . . . . # # # . . # . . # . # . . # . # . . . . . # # . .


day 11

In [ ]:
import os
import re
#import sys; sys.set_int_max_str_digits(10)

class Item(object):
  def __init__(self, worry_level):
    self.worry_level = worry_level

class Monkey(object):
  def __init__(self, id, items, operation, test):
    self.id = id
    self.n_inspections = 0
    self.items = items
    self.operation = operation
    self.test = test

def parse(fn):
  paragraphs = open(fn).read().split(os.linesep + os.linesep)
  monkeys = []
  for pg in paragraphs:
    pg = pg.split(os.linesep)
    id = int(re.findall('Monkey ([0-9]+)', pg[0])[0])
    items_str = pg[1].replace('  Starting items: ', '').split(', ')
    items = list(map(lambda worry_level: Item(int(worry_level)), items_str))
    operation = pg[2].replace('  Operation: new = old ', '')
    operation = operation.split(' ')
    test1 = int(re.findall('Test: divisible by ([0-9]+)', pg[3])[0])
    test2 = int(re.findall('If true: throw to monkey ([0-9]+)', pg[4])[0])
    test3 = int(re.findall('If false: throw to monkey ([0-9]+)', pg[5])[0])
    test = (test1, test2, test3)
    monkeys.append(Monkey(id, items, operation, test))
  return monkeys

def print_result(round, monkeys):
  print(f'After round {round}, the monkeys are holding items with these worry levels:')
  for monkey in monkeys:
    print(f'Monkey {monkey.id}: {[item.worry_level for item in monkey.items]}')
  print()

def p1(monkeys):
  ROUNDS = 20
  for round in range(ROUNDS):
    for monkey in monkeys:
      while monkey.items:
        item = monkey.items.pop(0)
        monkey.n_inspections += 1
        if monkey.operation[1] == 'old':
          item.worry_level = eval(f'{item.worry_level} {monkey.operation[0]} {item.worry_level}')
        else:
          item.worry_level = eval(f'{item.worry_level} {monkey.operation[0]} {monkey.operation[1]}')

        item.worry_level //= 3
        divisor, divisible_target, indivisible_target = monkey.test

        if item.worry_level % divisor == 0:
          monkeys[divisible_target].items.append(item)
        else:
          monkeys[indivisible_target].items.append(item)
    #print_result(round + 1, monkeys)

  inspections = sorted([m.n_inspections for m in monkeys], reverse=True)[:2]
  return inspections[0] * inspections[1]

p1(parse('11_1.txt'))

10605

In [ ]:
from math import gcd

def get_lcm(ls):
  lcm = 1
  for i in ls:
    lcm = lcm * i // gcd(lcm, i)
  return lcm

def p2(monkeys):
  DIVISOR_PRODUCT = 1
  for monkey in monkeys:
    DIVISOR_PRODUCT *= monkey.test[0]

  #divisors = [monkey.test[0] for monkey in monkeys]
  #LCM_DIVISORS = get_lcm(divisors)

  ROUNDS = 10000
  for round in range(ROUNDS):
    for monkey in monkeys:
      while monkey.items:
        item = monkey.items.pop(0)
        monkey.n_inspections += 1
        if monkey.operation[1] == 'old':
          item.worry_level = eval(f'{item.worry_level} {monkey.operation[0]} {item.worry_level}')
        else:
          item.worry_level = eval(f'{item.worry_level} {monkey.operation[0]} {monkey.operation[1]}')

        divisor, divisible_target, indivisible_target = monkey.test
        item.worry_level %= DIVISOR_PRODUCT # divisors are all primes?

        if item.worry_level % divisor == 0:
          monkeys[divisible_target].items.append(item)
        else:
          monkeys[indivisible_target].items.append(item)

  inspections = sorted([m.n_inspections for m in monkeys], reverse=True)[:2]
  return inspections[0] * inspections[1]

p2(parse('11_2.txt'))

15117269860

day 12

In [ ]:
from heapq import heappush, heappop

def parse(fn):
  grid = list(map(lambda row: list(row), open(fn).read().split('\n')[:-1]))
  nrows, ncols = len(grid), len(grid[0])
  start = [(r, c) for r in range(nrows) for c in range(ncols) if grid[r][c] == 'S'][0]
  end = [(r, c) for r in range(nrows) for c in range(ncols) if grid[r][c] == 'E'][0]
  grid[end[0]][end[1]] = 'z'
  return grid, start, end

def neighbors(grid, r, c):
  out = []
  for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
    x, y = r + dx, c + dy
    if x < len(grid) and x >= 0 and y < len(grid[0]) and y >= 0 and (x, y) != (r, c):
      out.append((x, y))
  return out

def p1(grid, start, end):
  visited = set([xy for xy in neighbors(grid, start[0], start[1])])
  q = [(1, xy, [start, xy]) for xy in neighbors(grid, start[0], start[1])]
  while q:
    nsteps, (r, c), path = heappop(q)
    if (r, c) == end:
      return nsteps

    for (x, y) in neighbors(grid, r, c):
      if ord(grid[x][y]) <= (ord(grid[r][c]) + 1) and (x, y) not in visited:
        heappush(q, (nsteps + 1, (x, y), path + [(x, y)]))
        visited.add((x, y))

  return 0

grid, start, end = parse('12_2.txt')
p1(grid, start, end)

456

In [ ]:
import math

def p2(grid, start, end):
  nrows, ncols = len(grid), len(grid[0])
  grid[start[0]][start[1]] = 'a'
  squares_marked_a = [(x, y) for x in range(nrows) for y in range(ncols) if grid[x][y] == 'a']

  visited = {end} # set([xy for xy in neighbors(grid, end[0], end[1])])
  q = [(0, end)] # [(1, xy) for xy in neighbors(grid, end[0], end[1])]
  cost = [[math.inf for _ in range(ncols)] for _ in range(nrows)]
  cost[end[0]][end[1]] = 0

  while q:
    nsteps, (r, c) = heappop(q)
    for (x, y) in neighbors(grid, r, c):
      if ord(grid[x][y]) >= (ord(grid[r][c]) - 1) and (x, y) not in visited:
        cost[x][y] = nsteps + 1
        heappush(q, (nsteps + 1, (x, y)))
        visited.add((x, y))

  m = nrows * ncols
  for (x, y) in squares_marked_a:
    m = min(m, cost[x][y])
  return m

grid, start, end = parse('12_2.txt')
p2(grid, start, end)

454